In [1]:
import plotly.express as px
import yfinance as yf
import polars as pl
from data_loading import preprocess_history
from model import Model
from huggingface_hub import HfApi
from dotenv import load_dotenv
import numpy as np

load_dotenv()

True

In [330]:
df = pl.read_parquet("test-stocks.parquet")

In [3]:
api = HfApi()
nn_model = Model.load_from_hf(api, 'Kicel/economics3a', 'itransformer')
nn_model.eval()

itransformer.bin:   0%|          | 0.00/7.42M [00:00<?, ?B/s]

c:\Users\Игорь\.pyenv\pyenv-win\versions\3.12.4\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Игорь\.cache\huggingface\hub\models--Kicel--economics3a. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [432]:
history = df.filter(symbol='GOOG')
nn_data = preprocess_history(history.select(pl.all().name.to_lowercase())).to_jax()

In [456]:
offset = 123
extend = 0
data = history[-32-8-offset:len(history)-offset]
forecast = nn_model.forecast(nn_data[-32-8-offset+extend:-8-offset+extend])[:8-extend]
pred = np.pad(
    np.hstack([
        data[-9, 'close'],
        forecast  
    ]),
    (31 + extend, 0),
    constant_values=np.nan
)
# pred_data = data.with_columns(pred=np.pad(np.array(forecast), (32 + extend, 0), constant_values=np.nan))
pred_data = data.with_columns(pred=pred)

In [457]:
fig = px.line(pred_data, x='date', y=['close', 'pred'])
fig.add_vline(pred_data[-8+extend, 'date'])
fig